In [1]:
import pandas as pd

Dataset:
https://portal.edirepository.org/nis/mapbrowse?packageid=edi.649.6

Metadata:
https://portal.edirepository.org/nis/metadataviewer?packageid=edi.649.6

More info:
https://marinemitigation.msi.ucsb.edu/data?field_species_target_id=&field_data_category_target_id%5B44%5D=44

These data describe annual estimates of bird density collected as part of the SONGS San Dieguito Wetland Restoration mitigation monitoring program designed to evaluate compliance of the restoration project with conditions of the SONGS permit. Monitoring began in 2012 in the San Dieguito Wetlands and Tijuana Estuary in San Diego County, CA, Carpinteria Salt Marsh in Santa Barbara County, CA, and Mugu Lagoon in Ventura County, CA. The abundance of birds is determined from **twenty plots spread across each wetland.**

CSM = Carpinteria Salt Marsh
MUL = Mugu Lagoon
SDW = San Dieguito Wetland
TJE = Tijuana Estuary

In [2]:
df = pd.read_csv('wetland_ts_bird_abundance-2024-06-12_14-38-10.csv')
df.head()

,year,survey,date,wetland_code,module_code,bird_plot_number,start_time,cloud_cover,temperature,wind_speed,wind_direction,precipitation_code,bird_count_visibility_code,species_id,species_code,genus_name,species_name,bird_flight_code,count,bird_plot_area_acres
0,2010,1,2010-02-03,CSM,CSM,1,04:04:00,-99999,60,-99999,NOT RECORDED,NONE,GOOD,9,COHA,Accipiter,cooperii,A,0,3.71
1,2010,1,2010-02-03,CSM,CSM,1,04:04:00,-99999,60,-99999,NOT RECORDED,NONE,GOOD,9,COHA,Accipiter,cooperii,G,0,3.71
2,2010,1,2010-02-03,CSM,CSM,1,04:04:00,-99999,60,-99999,NOT RECORDED,NONE,GOOD,13,SSHA,Accipiter,striatus,A,0,3.71
3,2010,1,2010-02-03,CSM,CSM,1,04:04:00,-99999,60,-99999,NOT RECORDED,NONE,GOOD,13,SSHA,Accipiter,striatus,G,0,3.71
4,2010,1,2010-02-03,CSM,CSM,1,04:04:00,-99999,60,-99999,NOT RECORDED,NONE,GOOD,20,SPSA,Actitis,macularius,A,0,3.71


In [3]:
# map months to seasons
def month_to_season(month):
    if month in [12, 1, 2]:
        return 'winter'
    elif month in [3, 4, 5]:
        return 'spring'
    elif month in [6, 7, 8]:
        return 'summer'
    elif month in [9, 10, 11]:
        return 'fall'

# add column with seasons
df['season'] = pd.to_datetime(df.date).dt.month.apply(month_to_season)

In [ ]:

# total bird counts by year, wetland, and season
seasonal = df.groupby(['year','wetland_code','season'])['count'].sum().reset_index()

# make columns = wetland codes x season pairs
pivot = seasonal.pivot(index='year', columns=['wetland_code','season'], values='count')

# flatten column names
pivot.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in pivot.columns]
pivot = pivot.reset_index()
pivot

In [4]:
# check: every year,wetland,season combination has the same number of species listed
# all species are listed even if they have 0 counts
df.groupby(['year','wetland_code','season'])['species_code'].nunique().nunique()

1

In [35]:
# instead, get number of different species seen per season
df[df['count']!=0].groupby(['year','wetland_code','season'])['species_code'].nunique().reset_index()

,year,wetland_code,season,species_code
0,2010,CSM,fall,50
1,2010,CSM,spring,40
2,2010,CSM,winter,39
3,2010,MUL,fall,61
4,2010,MUL,winter,45
...,...,...,...,...
152,2023,SDW,spring,63
153,2023,SDW,winter,64
154,2023,TJE,fall,38
155,2023,TJE,spring,56


In [36]:
# number of species per year, season, wetland registered
species = df[df['count']!=0].groupby(['year','wetland_code','season'])['species_code'].nunique().reset_index()

# make columns = wetland codes x season pairs
species = species.pivot(index='year', columns=['wetland_code','season'], values='species_code')

# flatten column names
species.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in species.columns]
species = species.reset_index()
species


,year,CSM_fall,CSM_spring,CSM_winter,MUL_fall,MUL_winter,SDW_fall,SDW_spring,TJE_fall,MUL_spring,SDW_winter,TJE_spring,TJE_winter
0,2010,50.0,40.0,39.0,61.0,45.0,85.0,75.0,81.0,NaN,NaN,NaN,NaN
1,2011,NaN,44.0,48.0,NaN,58.0,NaN,74.0,NaN,52.0,78.0,70.0,67.0
2,2012,49.0,43.0,51.0,53.0,57.0,73.0,72.0,69.0,58.0,71.0,63.0,70.0
3,2013,38.0,46.0,42.0,62.0,60.0,70.0,70.0,64.0,58.0,69.0,74.0,69.0
4,2014,45.0,43.0,38.0,57.0,49.0,71.0,78.0,62.0,52.0,61.0,81.0,60.0
5,2015,45.0,42.0,44.0,51.0,58.0,65.0,61.0,64.0,50.0,71.0,76.0,73.0
6,2016,47.0,36.0,41.0,58.0,63.0,57.0,62.0,58.0,48.0,67.0,76.0,76.0
7,2017,43.0,41.0,46.0,53.0,57.0,54.0,45.0,57.0,54.0,66.0,63.0,72.0
8,2018,44.0,48.0,48.0,57.0,56.0,51.0,49.0,55.0,54.0,55.0,60.0,66.0
9,2019,40.0,39.0,39.0,53.0,57.0,53.0,55.0,50.0,52.0,54.0,54.0,63.0


In [ ]:
df.groupby(['year','wetland_code','survey'])['count'].sum()
df[(df.year==2010) & (df.wetland_code == 'CSM') & (df.survey == 1)]['count'].sum()

subset = df[(df.year==2010) & (df.wetland_code == 'SDW') & (df.season == 'fall') & (df['count'] != 0)]
print(len(subset.bird_plot_number.unique()))
print(subset.groupby('bird_plot_number')['count'].sum())
subset.sort_values(by='count', ascending=False).head()

In [5]:
abundance = pd.read_csv('wetland_ps_bird_abundance_and_richness-2024-06-12_14-47-54.csv')
abundance.head()

,year,wetland_code,module_code,bird_plot_number,bird_count_per_plot,bird_species_richness_per_plot
0,2010,CSM,CSM,1,3.2857,11
1,2010,CSM,CSM,2,3.7143,11
2,2010,CSM,CSM,3,2.7857,12
3,2010,CSM,CSM,4,4.5000,18
4,2010,CSM,CSM,5,63.4286,38


In [6]:
abundance.wetland_code.unique()

array(['CSM', 'MUL', 'SDW', 'TJE'], dtype=object)

In [14]:
# checking plot numbers are not repeated in a year
for year in range(2010,2024):
 plot_num = len(abundance.loc[(abundance.wetland_code == 'SDW') & (abundance.year == 2013),'bird_plot_number'].unique())
 n = len(abundance[(abundance.wetland_code == 'SDW') & (abundance.year == 2013)])
 if n != plot_num:
    print("doesn't match")
 